In [ ]:
pip install zstandard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/kmrasmussen/pythia_tools.git pythia_tools
!pip install -e pythia_tools
!pip install git+https://github.com/neelnanda-io/TransformerLens.git --quiet

Cloning into 'pythia_tools'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 145 (delta 62), reused 73 (delta 25), pack-reused 34
Receiving objects: 100% (145/145), 57.73 MiB | 11.75 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Obtaining file:///content/pythia_tools
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 

In [ ]:
import zstandard as zstd
import json
import io
import pandas as pd
from pythia_tools.models import get_tokenizer
from tqdm import tqdm
import torch
from pythia_tools.utils import mem_size

In [ ]:
tokenizer = get_tokenizer()

In [ ]:
input_path = '/content/drive/MyDrive/thesis/data/the_pile_train/chunk_1.json.zst' #'/content/drive/MyDrive/thesis/data/val.jsonl.zst'
#output_path = '/content/drive/MyDrive/thesis/data/val100k_3k.jsonl'
#extract_lines(input_path, output_path, 100000, 3000)

In [ ]:
max_seqs

300000

In [ ]:
def get_T_from_zst_path(input_path, num_lines, line_cap, max_seqs):
  dctx = zstd.ZstdDecompressor()
  seqs = []
  with open(input_path, "rb") as input_file:
    # Create a decompression stream reader
    with dctx.stream_reader(input_file) as reader:
      # Wrap the reader with a buffered reader to read lines
      buffered_reader = io.BufferedReader(reader)
      for i in tqdm(range(num_lines)):
        line = buffered_reader.readline()
        if not line:
          break
        json_object = json.loads(line)
        if line_cap is not None: # and len(json_object) > line_cap:
          #print('capping')
          json_object['text'] = json_object['text'][:line_cap]
        line_text = json_object['text']
        line_enc = tokenizer.encode(line_text)
        #print(i, len(line_text), json_object['meta'], len(line_enc))
        if len(line_enc) >= 600:
          seqs.append(line_enc[:600])
          if len(seqs) % 10000 == 0:
            print(len(seqs))
        #if len(seqs) >= max_seqs:
        #  break
  T = torch.stack([torch.tensor(seq) for seq in seqs])
  return T

In [ ]:
num_lines = 10000000
line_cap = 3000
max_seqs = 300000

In [ ]:
for i in [11] #range(12,21):
  print(i)
  input_path = f'/content/drive/MyDrive/thesis/data/the_pile_train/chunk_{i}.json.zst'
  output_path = f'/content/drive/MyDrive/thesis/data/the_pile_train_Ts/chunk_{i}.pt' # '/content/drive/MyDrive/thesis/data/token_mats/val_300k_600.pt'
  T = get_T_from_zst_path(input_path, num_lines, line_cap, max_seqs)
  torch.save(T, output_path)

12


  0%|          | 27884/10000000 [01:17<5:45:21, 481.25it/s]

10000


  1%|          | 55650/10000000 [02:26<6:01:50, 458.04it/s]

20000


...

  0%|          | 27418/10000000 [01:12<8:58:00, 308.93it/s]

10000


  1%|          | 55214/10000000 [02:21<8:05:33, 341.36it/s]

20000


  1%|          | 80609/10000000 [03:28<7:06:44, 387.41it/s]


# Concat chunk tensors

In [ ]:
Ts_dict = {}
for i in range(0,11):
  print(i)
  chunk_T_path = f'/content/drive/MyDrive/thesis/data/the_pile_train_Ts/chunk_{i}.pt' # '/content/drive/MyDrive/thesis/data/token_mats/val_300k_600.pt'
  T = torch.load(chunk_T_path)
  Ts_dict[i] = T

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
for chunk_id, X in Ts_dict.items():
  print(chunk_id, X.shape)

0 torch.Size([29828, 600])
1 torch.Size([28445, 600])
2 torch.Size([29889, 600])
3 torch.Size([29012, 600])
4 torch.Size([28765, 600])
5 torch.Size([29920, 600])
6 torch.Size([29740, 600])
7 torch.Size([29991, 600])
8 torch.Size([29702, 600])
9 torch.Size([27927, 600])
10 torch.Size([29554, 600])


In [ ]:
Ts_list = [X for _, X in Ts_dict.items()]

In [ ]:
Ts_tensor = torch.cat(Ts_list, dim=0)

In [ ]:
Ts_tensor.shape

torch.Size([322773, 600])

In [ ]:
torch.save(Ts_tensor, '/content/drive/MyDrive/thesis/data/token_mats/train_322k_600.pt')

# Old

In [ ]:
def extract_lines(input_path, output_path, num_lines, line_cap=None):
    # Create a zstandard decompression context
    dctx = zstd.ZstdDecompressor()

    data = []

    # Open the compressed input file
    with open(input_path, "rb") as input_file:
        # Create a decompression stream reader
        with dctx.stream_reader(input_file) as reader:
            # Wrap the reader with a buffered reader to read lines
            buffered_reader = io.BufferedReader(reader)

            # Open the output file for writing
            with open(output_path, "w") as output_file:
                # Read and process the specified number of lines
                for _ in range(num_lines):
                    line = buffered_reader.readline()
                    if not line:
                        break  # Stop reading if the end of the file is reached

                    # Load the JSON object from the line
                    json_object = json.loads(line)
                    if line_cap is not None: # and len(json_object) > line_cap:
                      print('capping')
                      json_object['text'] = json_object['text'][:line_cap]
                    else:
                      print('no line cap')
                    print(len(json_object['text']), json_object['meta'])

                    # Write the JSON object to the output file as a JSON line
                    output_file.write(json.dumps(json_object) + "\n")

In [ ]:
def jsonl_to_dataframe(input_path):
    # Read the input .jsonl file
    data = []
    with open(input_path, "r") as input_file:
        for line in input_file:
            # Load the JSON object from the line
            json_object = json.loads(line)

            # Extract the text and meta fields
            text = json_object["text"]
            meta = json_object["meta"]['pile_set_name']

            # Append the data to the list
            data.append({"text": text, "meta": meta})

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    return df

In [ ]:
output_csv_path = '/content/drive/MyDrive/thesis/data/val100k_3k.csv'

In [ ]:
df = jsonl_to_dataframe(output_csv_path)

FileNotFoundError: ignored

In [ ]:
df.to_csv(output_csv_path, index=False)

NameError: ignored